Here I tried to use Light AutoML library developed by Sber. But as competition rules prohibits use of internet, we can't use it in submission.
So play with it and see what results you can get!

In [ ]:
!pip install -U lightautoml

In [ ]:
import traceback
import numpy as np
import pandas as pd
import datatable as dt
import gresearch_crypto

path = "../input/g-research-crypto-forecasting/"
df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "example_test.csv")
df_asset_details = pd.read_csv(path + "asset_details.csv")
df_supp_train = pd.read_csv(path + "supplemental_train.csv")

In [ ]:
df_train.head()

In [ ]:
df_asset_details

In [ ]:
def hlco_ratio(df): return (df['High'] - df['Low'])/(df['Close'] - df['Open'])
def upper_shadow(df): return df['High'] - np.maximum(df['Close'], df['Open'])
def lower_shadow(df): return np.minimum(df['Close'], df['Open']) - df['Low']

def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Target', 'Asset_ID', 'timestamp']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    df_feat['hlco_ration'] = hlco_ratio(df_feat)
    return df_feat


In [ ]:
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train = df_train.dropna(how="any")

train_data = get_features(df_train)
train_data = df_train.sample(frac = 0.8)
valid_data = df_train.drop(train_data.index)

In [ ]:
valid_data.head()

In [ ]:
btc_data.head()

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from scipy.spatial.distance import correlation
import scipy

automl = {}

def metric(y_true, y_pred):
    print(np.std(y_pred))
    if np.std(y_pred) == 0.:
        return 0
    p = scipy.stats.pearsonr(y_true, y_pred)[0]
    print(f'pearson {p}')
    return p

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    automl[asset_id] = TabularAutoML(
        task = Task(
            name = 'reg')
    #     general_params = {'nested_cv': False},
    )

    btc_data = train_data[train_data['Asset_ID'] == asset_id]

    oof_pred = automl[asset_id].fit_predict(
        btc_data,
        roles = {'target': 'Target', 'drop': ['timestamp', 'Asset_ID']},
        valid_data = valid_data[valid_data['Asset_ID'] == asset_id]
    )
# test_pred = automl[asset_id].predict(valid_data)

In [ ]:
# from lightautoml.automl.presets.tabular_presets import TabularAutoML
# from lightautoml.tasks import Task
# from scipy.spatial.distance import correlation

# automl = {}

# for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
#     automl[asset_id] = TabularAutoML(
#         task = Task(
#             name = 'reg',
#             general_params = {'nested_cv': False, 'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
#             metric = lambda y_true, y_pred: correlation(y_true, (y_pred > 0.5)*1))
#     )
#     oof_pred = automl[asset_id].fit_predict(
#         train_data[train_data['Asset_ID'] == asset_id],
#         roles = {'target': 'Target', 'drop': ['timestamp', 'Asset_ID']}
#     )
#     test_pred = automl[asset_id].predict(valid_data)

In [ ]:
# valid_target = valid_data["Target"].to_numpy()
# preds = [n.data[0] for n in test_pred]

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12,6))
# plt.plot(valid_target)
# plt.plot(preds)
# plt.show()

In [ ]:
x_test = valid_data.iloc[2].drop('Target')
y_pred = automl[1].predict(pd.DataFrame([x_test]))[0].data[0]
print(x_test)
print(y_pred)

## Score your model

In [ ]:
corr = np.corrcoef(preds, valid_target)
print('Test score for BTC: ', f"{corr[0,1]:.4f}")
import scipy
corr_p = scipy.stats.pearsonr(preds, valid_target)
corr_p[0]

# Results Submission

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():        
        if automl[row['Asset_ID']] is not None:
            try:
                model = automl[row['Asset_ID']]
                x_test = get_features(row)
                y_pred = model.predict(pd.DataFrame([x_test]))[0].data[0]
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
            except:
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
                traceback.print_exc()
        else: 
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0        
    env.predict(df_pred)